In [1]:
from dash import Dash, html, dcc, callback, Input, Output, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash_bootstrap_templates import load_figure_template

In [112]:
# Load the data
athlete_events = pd.read_csv('Data/athlete_events.csv')
athlete_events.sort_values(by='Year', inplace=True)
athlete_events.head()

medal_gender_counts = athlete_events[['Sex', 'Sport', 'Medal']].groupby(['Sport']).get_group('Athletics').dropna()
medal_gender_counts.head(10)

medal_counts_men = medal_gender_counts[medal_gender_counts['Sex'] == 'M'].value_counts().sum()
medal_counts_women = medal_gender_counts[medal_gender_counts['Sex'] == 'F'].value_counts().sum()
medal_counts_women

1275

In [121]:
load_figure_template("flatly")

app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1('Olympic Games', style={'fontSize': '30px', 'border-width': '0px', 'border-style': 'solid', 'border-radius': '20px', 'box-shadow': '0px 0px 14px 5px rgba(32,73,179,0.75)', 'text-align': 'center', 'padding-top': '10px', 'padding-bottom': '10px'}),
            html.H3('Year', style={'fontSize': '24px'}, className="mt-2"),
            dcc.Dropdown(
                id='year_dropdown',
                options=[{'label': i, 'value': i} for i in athlete_events['Year'].unique()],
                value=None,
                style={'width': '75%'}
            ),
            html.H3('Nation Code', style={'fontSize': '24px'}, className="mt-2"),
            dcc.Dropdown(
                id='country_dropdown',
                options=[{'label': i, 'value': i} for i in athlete_events['NOC'].unique()],
                value=None,
                style={'width': '75%'}
            ),
            html.H3('Sport', style={'fontSize': '24px'}, className="mt-2"),
            dcc.Dropdown(
                id='sport_dropdown',
                options=[{'label': i, 'value': i} for i in athlete_events['Sport'].unique()],
                value=None,
                style={'width': '75%'}
            ),
        ], width=3),

        dbc.Col([
            html.Div(
                [html.H3('No. of competitors:', style={'fontSize': '20px', }),
                html.H3(id="no_competitors")],
                style={'border-color': 'black', 'border-width': '0px', 'border-style': 'solid', 'border-radius': '20px', 'box-shadow': '0px 0px 14px 5px rgba(32,73,179,0.75)', 'text-align': 'center', 'margin-right': '10px'}
            ),
            html.Div(
                [html.H3('No. of Medals:', style={'fontSize': '20px'}),
                html.H3(id="no_medals")],
                style={'border-color': 'black', 'border-width': '0px', 'border-style': 'solid', 'border-radius': '20px', 'box-shadow': '0px 0px 14px 5px rgba(32,73,179,0.75)', 'text-align': 'center', 'margin-top': '10px', 'margin-right': '10px'}
            )
        ], width=2),

        dbc.Col([
            html.H3('Medal Breakdown By Top 10 Sports & Type', style={'fontSize': '16px'}),
            dcc.Graph(id='medal_breakdown_sport')
        ], width=3, style={'border-color': 'black', 'border-width': '0px', 'border-style': 'solid', 'border-radius': '20px', 'box-shadow': '0px 0px 14px 5px rgba(32,73,179,0.75)', 'margin-right': 'auto'}),

        dbc.Col([
            html.H3('Medal breakdown By Top 10 Athlete & Type', style={'fontSize': '16px'}),
            dcc.Graph(id='medal_breakdown_athlete')
        ], width=3, style={'border-color': 'black', 'border-width': '0px', 'border-style': 'solid', 'border-radius': '20px', 'box-shadow': '0px 0px 14px 5px rgba(32,73,179,0.75)', 'margin-right': 'auto'}),    
    ], justify='center', style={'margin-top': '20px'}),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='pie_chart_competitor_gender', style={'margin-top': '20px'}),
        ], width=3),

        dbc.Col([
            dcc.Graph(id="pie_chart_medal_gender", style={'margin-top': '20px'}),
        ], width=3),

        dbc.Col([

        ], width=1),

        # dbc.Col([
        #     dcc.Graph(id="medal_distribution_over_time", style={'margin-top': '20px'}),
        # ], width=5),  
    ])
], fluid=True)

@callback(
    Output('no_competitors', 'children'),
    Output('no_medals', 'children'),
    Output('medal_breakdown_sport', 'figure'),
    Output('medal_breakdown_athlete', 'figure'),
    Output('pie_chart_competitor_gender', 'figure'),
    Output('pie_chart_medal_gender', 'figure'),
    # Output('medal_distribution_over_time', 'figure'),
    Input('year_dropdown', 'value'),
    Input('country_dropdown', 'value'),
    Input('sport_dropdown', 'value'),
)

def update_graph(year, country, sport):
    if year is None and country is None and sport is None:
        no_of_competitors = athlete_events['Name'].nunique()  # Sum of all years and countries combined

        no_of_medals = athlete_events['Medal'].count()

        medal_breakdown = athlete_events.groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events.groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)

        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))

        unique_competitors = athlete_events[['Name', 'Sex']].drop_duplicates()
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')
        

        medal_gender_counts = athlete_events.groupby(['Sex', 'Medal'])['Medal'].count().reset_index(name='Count')

        # Filter the counts for women and men
        women_counts = medal_gender_counts[medal_gender_counts['Sex'] == 'F']
        men_counts = medal_gender_counts[medal_gender_counts['Sex'] == 'M']

        # Create a pie chart for the proportion of medals won by women and men
        pie_chart_medals = px.pie(names=['Women', 'Men'], values=[women_counts['Count'].sum(), men_counts['Count'].sum()], title='Proportion of Medals Won by Women and Men')

        # Show the percentage labels on the pie chart
        pie_chart_medals.update_traces(textposition='inside', textinfo='percent+label')

    elif year is None and country is None:
        no_of_competitors = athlete_events.groupby('Sport').get_group(sport)['Name'].nunique()

        no_of_medals = athlete_events.groupby('Sport').get_group(sport)['Medal'].count()

        medal_breakdown = athlete_events[athlete_events['Sport'] == sport].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events[athlete_events['Sport'] == sport].groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)

        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))

        unique_competitors = athlete_events[['Name', 'Sex', 'Sport']].drop_duplicates().groupby('Sport').get_group(sport)
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')

        medal_gender_counts = athlete_events[['Sex', 'Sport', 'Medal']].groupby(['Sport']).get_group(sport).dropna()

        # Filter the counts for women and men
        medal_counts_men = medal_gender_counts[medal_gender_counts['Sex'] == 'M'].value_counts().sum()
        medal_counts_women = medal_gender_counts[medal_gender_counts['Sex'] == 'F'].value_counts().sum()

        # Create a pie chart for the proportion of medals won by women and men
        pie_chart_medals = px.pie(names=['Women', 'Men'], values=[medal_counts_women, medal_counts_men], title='Proportion of Medals Won by Women and Men')

        # Show the percentage labels on the pie chart
        pie_chart_medals.update_traces(textposition='inside', textinfo='percent+label')
        
    elif year is None and sport is None:
        no_of_competitors = athlete_events.groupby('NOC').get_group(country)['Name'].nunique()

        no_of_medals = athlete_events.groupby('NOC').get_group(country)['Medal'].count()
        
        medal_breakdown = athlete_events[athlete_events['NOC'] == country].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)

        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events[athlete_events['NOC'] == country].groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)

        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))
        
        unique_competitors = athlete_events[['Name', 'Sex', 'NOC']].drop_duplicates().groupby('NOC').get_group(country)
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')

        medal_gender_counts = athlete_events[['Sex', 'NOC', 'Medal']].groupby(['NOC']).get_group(country).dropna()

        # Filter the counts for women and men
        medal_counts_men = medal_gender_counts[medal_gender_counts['Sex'] == 'M'].value_counts().sum()
        medal_counts_women = medal_gender_counts[medal_gender_counts['Sex'] == 'F'].value_counts().sum()

        # Create a pie chart for the proportion of medals won by women and men
        pie_chart_medals = px.pie(names=['Women', 'Men'], values=[medal_counts_women, medal_counts_men], title='Proportion of Medals Won by Women and Men')

        # Show the percentage labels on the pie chart
        pie_chart_medals.update_traces(textposition='inside', textinfo='percent+label')

    elif country is None and sport is None:
        no_of_competitors = athlete_events.groupby('Year').get_group(year)['Name'].nunique()

        no_of_medals = athlete_events.groupby('Year').get_group(year)['Medal'].count()

        medal_breakdown = athlete_events[athlete_events['Year'] == year].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events[athlete_events['Year'] == year].groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))

        unique_competitors = athlete_events[['Name', 'Sex', 'Year']].drop_duplicates().groupby('Year').get_group(year)
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')

        medal_gender_counts = athlete_events[['Sex', 'Year', 'Medal']].groupby(['Year']).get_group(year).dropna()

        # Filter the counts for women and men
        medal_counts_men = medal_gender_counts[medal_gender_counts['Sex'] == 'M'].value_counts().sum()
        medal_counts_women = medal_gender_counts[medal_gender_counts['Sex'] == 'F'].value_counts().sum()

        # Create a pie chart for the proportion of medals won by women and men
        pie_chart_medals = px.pie(names=['Women', 'Men'], values=[medal_counts_women, medal_counts_men], title='Proportion of Medals Won by Women and Men')

        # Show the percentage labels on the pie chart
        pie_chart_medals.update_traces(textposition='inside', textinfo='percent+label')

    elif country is None:
        no_of_competitors = athlete_events.groupby('Year').get_group(year)['Name'].nunique()

        no_of_medals = athlete_events.groupby('Year').get_group(year)['Medal'].count()

        medal_breakdown = athlete_events[athlete_events['Year'] == year].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events[athlete_events['Year'] == year].groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)

        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))

        unique_competitors = athlete_events[['Name', 'Sex', 'Sport', 'Year']].drop_duplicates().groupby('Sport').get_group(sport).groupby('Year').get_group(year)
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')

    elif sport is None:
        no_of_competitors = athlete_events.groupby('NOC').get_group(country)['Name'].nunique()

        no_of_medals = athlete_events.groupby('NOC').get_group(country)['Medal'].count()

        medal_breakdown = athlete_events[athlete_events['NOC'] == country].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events[athlete_events['NOC'] == country].groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True)

        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))

        unique_competitors = athlete_events[['Name', 'Sex', 'NOC', 'Year']].drop_duplicates().groupby('NOC').get_group(country).groupby('Year').get_group(year)
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')

    elif year is None:
        no_of_competitors = athlete_events.groupby('NOC').get_group(country)['Name'].nunique()

        no_of_medals = athlete_events.groupby('NOC').get_group(country)['Medal'].count()

        medal_breakdown = athlete_events[athlete_events['NOC'] == country].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events[athlete_events['NOC'] == country].groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True)

        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))

        unique_competitors = athlete_events[['Name', 'Sex', 'NOC', 'Sport']].drop_duplicates().groupby('NOC').get_group(country).groupby('Sport').get_group(sport)
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')

    else:
        no_of_competitors = athlete_events[(athlete_events['Year'] == year) & (athlete_events['NOC'] == country)]['Name'].nunique()

        no_of_medals = athlete_events[(athlete_events['Year'] == year) & (athlete_events['NOC'] == country)]['Medal'].count()

        medal_breakdown = athlete_events[(athlete_events['Year'] == year) & (athlete_events['NOC'] == country)].groupby(['Sport', 'Medal'])['Medal'].count().reset_index(name='Count')
        medal_breakdown = medal_breakdown.groupby(['Sport', 'Medal'])['Count'].sum().reset_index(name='Count')
        # Get the top 10 sports with the most medals won
        top_10_sports = medal_breakdown.groupby('Sport')['Count'].sum().nlargest(10).index
        top_10_sports_medal_breakdown = medal_breakdown[medal_breakdown['Sport'].isin(top_10_sports)]
        # Add a column for the sum of all three medal types
        top_10_sports_medal_breakdown['Total'] = top_10_sports_medal_breakdown.groupby('Sport')['Count'].transform('sum')
        top_10_sports_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_sports = px.bar(top_10_sports_medal_breakdown, y='Sport', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_sports.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_sports.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_sports.update_traces(marker_color='gold', selector=dict(name='Gold'))

        athlete_medal_count = athlete_events[(athlete_events['Year'] == year) & (athlete_events['NOC'] == country)].groupby(['Name', 'Medal'])['Medal'].count().reset_index(name='Count')
        # Get the top 10 athletes with the most medals won
        top_10_athletes = athlete_medal_count.groupby('Name')['Count'].sum().nlargest(10).index
        top_10_athletes_medal_breakdown = athlete_medal_count[athlete_medal_count['Name'].isin(top_10_athletes)]
        # Add a column for the sum of all three medal types
        top_10_athletes_medal_breakdown['Total'] = top_10_athletes_medal_breakdown.groupby('Name')['Count'].transform('sum')
        top_10_athletes_medal_breakdown.sort_values(by='Total', ascending=True, inplace=True)
        
        fig_athletes = px.bar(top_10_athletes_medal_breakdown, y='Name', x='Count', color='Medal', barmode='stack', orientation='h')
        fig_athletes.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
        fig_athletes.update_traces(marker_color='silver', selector=dict(name='Silver'))
        fig_athletes.update_traces(marker_color='gold', selector=dict(name='Gold'))


        unique_competitors = athlete_events[['Name', 'Sex', 'NOC', 'Year', 'Sport']].drop_duplicates().groupby('NOC').get_group(country).groupby('Year').get_group(year).groupby('Sport').get_group(sport)
        sex_counts = unique_competitors['Sex'].value_counts()
        pie_chart_comptetitors = px.pie(names=['Women', 'Men'], values=sex_counts.values, title="Competitors by gender")
        pie_chart_comptetitors.update_traces(textposition='inside', textinfo='percent+label')

    return no_of_competitors, no_of_medals, fig_sports, fig_athletes, pie_chart_comptetitors, pie_chart_medals

app.run(debug=True, jupyter_mode='external')


Dash app running on http://127.0.0.1:8050/


/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_64752/356000506.py:99: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_64752/356000506.py:100: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_64752/356000506.py:112: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta